## 오토인코더
- **인코더**의 역할을 맡아 각 의류 아이템을 옷장의 특정 위치로 이동하며 이 과정을 **인코딩**이라 한다.
- 브라이언은 **디코더**의 역할을 맡아 옷장의 한 위치를 받아 해당 아이템을 다시 생성하려고 시도한다. 이 과정을 **디코딩**이라 한다.

In [11]:
# 패션 MNIST 데이터셋 로드

from tensorflow.keras import datasets, layers, models
import tensorflow.keras.backend as K
import numpy as np
(x_train, y_train), (x_test, y_test) = datasets.fashion_mnist.load_data()

In [12]:
# 데이터 로드
(x_train, y_train), (x_test, y_test) = datasets.fashion_mnist.load_data()

In [13]:
# 데이터 전처리
def preprocess(imgs):
    imgs = imgs.astype("float32") / 255.0
    imgs = np.pad(imgs, ((0,0), (2,2), (2,2)), constant_values=0.0)
    imgs = np.expand_dims(imgs, -1)
    return imgs

x_train = preprocess(x_train)
x_test = preprocess(x_test)

- 원본 이미지는 28X28 크기 흑백 이미지(픽셀 값은 0~255)이므로 픽셀 값을 0~1 사이로 조정해야 한다.
- 또한 위 예제처럼 이미지가 신경망을 통과할 때 텐서 크기를 쉽게 조작할 수있도록 각 이미지에 패딩을 추가하여 32X32 크기로 만든다.

In [14]:
#1 인코더의 Input 층(이미지)을 정의
encoder_input = layers.Input(
	shape=(32, 32,1), name = "encoder_input"
)
#2 순서대로 Conv2D 층을 쌓는다.
x = layers.Conv2D(32, (3, 3), strides = 2, activation = 'relu', padding="same")(
	encoder_input
)
x = layers.Conv2D(64, (3, 3), strides = 2, activation = 'relu', padding="same")(x)
x = layers.Conv2D(128, (3, 3), strides = 2, activation = 'relu', padding="same")(x)
shape_before_flattening = K.int_shape(x)[1:]

#3 마지막 합성곱 층의 출력을 벡터로 펼침
x = layers.Flatten()(x)
#4 이 벡터를 2D 임베딩에 해당하는 Dense 층에 연결
encoder_output = layers.Dense(2, name="encoder_output")(x)

#5 케라스 Model 클래스로 인코더를 정의. 이 모델은 입력 이미지를 받아 이를 2D 임베딩에 인코딩한다.
encoder = models.Model(encoder_input, encoder_output)

In [15]:
encoder.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_input (InputLayer)  [(None, 32, 32, 1)]       0         
                                                                 
 conv2d_3 (Conv2D)           (None, 16, 16, 32)        320       
                                                                 
 conv2d_4 (Conv2D)           (None, 8, 8, 64)          18496     
                                                                 
 conv2d_5 (Conv2D)           (None, 4, 4, 128)         73856     
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 encoder_output (Dense)      (None, 2)                 4098      
                                                                 
Total params: 96770 (378.01 KB)
Trainable params: 96770 (378.

In [16]:
#1 디코더의 Input층 (임베딩)을 정의
decoder_input = layers.Input(shape=(2,), name="decoder_input")

#2 입력을 Dense 층에 연결
x = layers.Dense(np.prod(shape_before_flattening))(decoder_input)
#3 첫 번째 Conv2Dtranspose 층에 입력으로 주입할 수 있도록 Reshape 층의 벡터의 크기를 바꾼다.
x = layers.Reshape(shape_before_flattening)(x)
#4 Conv2DTranspose 층을 연속으로 쌓음
x = layers.Conv2DTranspose(128, (3,3), strides=2, activation = 'relu', padding="same")(x)
x = layers.Conv2DTranspose(64, (3,3), strides=2, activation = 'relu', padding="same")(x)
x = layers.Conv2DTranspose(32, (3,3), strides=2, activation = 'relu', padding="same")(x)
decoder_output = layers.Conv2D(
	1,
	(3,3),
	strides = 1,
	activation="sigmoid",
	padding="same",
	name="decoder_output"
)(x)

#5 케라스 Model 클래스로 디코더를 정의
#5 이 모델은 잠재 공간의 임베딩을 받아 원본 이밎 도메인으로 디코
decoder = models.Model(decoder_input, decoder_output)

In [17]:
decoder.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 decoder_input (InputLayer)  [(None, 2)]               0         
                                                                 
 dense (Dense)               (None, 2048)              6144      
                                                                 
 reshape (Reshape)           (None, 4, 4, 128)         0         
                                                                 
 conv2d_transpose (Conv2DTr  (None, 8, 8, 128)         147584    
 anspose)                                                        
                                                                 
 conv2d_transpose_1 (Conv2D  (None, 16, 16, 64)        73792     
 Transpose)                                                      
                                                                 
 conv2d_transpose_2 (Conv2D  (None, 32, 32, 32)        1846

In [20]:
#1 케라스 Model 클래스로 완전한 오토인코더를 정의
#1 이 모델은 이미지를 입력으로 받아 인코더와 디코더를 통과시켜 원본 이미지의 재구성을 생성
autoencoder = models.Model(encoder_input, decoder(encoder_output))
autoencoder.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_input (InputLayer)  [(None, 32, 32, 1)]       0         
                                                                 
 conv2d_3 (Conv2D)           (None, 16, 16, 32)        320       
                                                                 
 conv2d_4 (Conv2D)           (None, 8, 8, 64)          18496     
                                                                 
 conv2d_5 (Conv2D)           (None, 4, 4, 128)         73856     
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 encoder_output (Dense)      (None, 2)                 4098      
                                                                 
 model_1 (Functional)        (None, 32, 32, 1)         2462